In [1]:
import pandas as pd
import dill

# Import DEW 2019 Aqueous Species

**To do:**
- Change the code to use a more obvious folder name than 'working'

This notebook runs the code required to generate a database from a set of DEW parameters.

Aaron Wolf wrote functions that make this process easier to implement, originally for use in his carbonated mantle demonstration. The following line runs this file, therefore defining the functions written within it.

In [2]:
%run core.ipynb

/Users/simonmatthews/opt/anaconda3/lib/python3.7/site-packages/rubicon/objc/ctypes_patch.py:24: UserWarning: rubicon.objc.ctypes_patch has only been tested with Python 3.4 through 3.6. The current version is sys.version_info(major=3, minor=7, micro=6, releaselevel='final', serial=0). Most likely things will work properly, but you may experience crashes if Python's internals have changed significantly.
  .format(sys.version_info)
/Users/simonmatthews/opt/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


The HKF equations are developed in the previous notebook and setup there as a coder module. This is a pre-requisite for this notebook, and so this line runs the previous notebook.

In [3]:
%run HKF_Equations.ipynb

## Import and process the DEW database

I have extracted the aqueous species parameters from the DEW "Aqueous Species Options" worksheet. I renamed some of the complexes so that conventions are applied more consistently. I added a column expressing the formulae in the format require by ENKI. For complexes where their parameters are estimated, this estimation must be done by the coder module, I do not import the DEW spreadsheet calculated estimates.

In [4]:
dew = pd.read_csv('dew2019_cleaned.csv')
dew = dew.fillna(0)
dew

,name,symbol,formula,G_ref,H_ref,S_ref,V_ref,Cp_ref,omega0,z,...,prd_ac,compl,gas,a1,a2,a3,a4,c1,c2,comments
0,"acetate,aq",CH3COO-,C(2)H(3)O(2),-88270,-116180.0,20.60,40.499926,6.311743,1.3182,-1,...,0,0,0,10.018000,2.809000,6.720000,-2.894100,26.300000,-3.860000,"revised January 26th, 2016; new a1 value from ..."
1,"acetic-acid,aq",CH3COOH,C(2)H(4)O(2),-94760,-116100.0,42.70,52.010022,41.083552,-0.0760,0,...,0,0,0,11.500000,5.500000,1.670000,-2.870000,44.900000,-2.630000,Plyasunov & Shock (2001)
2,Ag+,Ag(+),Ag(1),18427,25275.0,17.54,-0.800078,7.899675,0.2160,1,...,1,0,0,2.046420,-4.387714,7.018169,-2.597612,12.786274,-1.425436,Shock & Helgeson (1988) revised with new predi...
3,"AgCl,aq",AgCl(0),Ag(1)Cl(1),-17450,-18270.0,34.10,27.000000,6.700094,-0.0380,0,...,1,1,0,6.745843,2.139576,3.044087,-2.867450,9.743185,-1.669791,Sverjensky et al. (1997) with new V greater th...
4,AgCl2-,AgCl2(-),Ag(1)Cl(2),-51560,-61130.0,47.00,53.000000,7.699896,0.9169,-1,...,1,1,0,12.253325,7.383245,-1.352680,-3.084223,19.126467,-1.466131,Sverjensky et al. (1997) with new V greater th...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,Yb+3,Yb(+3),Yb(1),-153000,-160300.0,-56.90,-44.499994,-36.399634,2.4443,3,...,1,0,0,-9.223325,-1.432516,14.532520,-2.719780,7.353108,-10.449205,Shock & Helgeson (1988) revised with new predi...
224,Zn+2,Zn(+2),Zn(1),-35200,-36660.0,-26.20,-24.300094,-6.300282,1.4574,2,...,1,0,0,-1.417424,-9.478710,9.977699,-2.387150,15.901037,-4.317967,Shock & Helgeson (1988) revised with new predi...
225,ZnCl+,ZnCl(+),Zn(1)Cl(1),-66850,-66240.0,23.00,4.800000,19.899909,0.2025,1,...,1,1,0,2.549738,-1.855533,6.393947,-2.702292,19.694777,1.019011,Sverjensky et al. (1997) with revised a1 equal...
226,"ZnCl2,aq",ZnCl2(0),Zn(1)Cl(2),-98300,-109080.0,27.03,26.000000,34.660101,-0.0380,0,...,1,1,0,6.551633,1.954668,3.199130,-2.859806,26.129473,4.025663,Sverjensky et al. (1997) with revised a1 equal...


The parameters given in the DEW spreadsheet are multiplied by factors of 10 for easier display, and the units of energy are colories not Joules. This can be fixed here:

In [5]:
multipliers = {'a1': 4.184/10,
               'a2': 4.184*100,
               'a3': 4.184,
               'a4': 4.184*1e4,
               'c1': 4.184,
               'c2': 4.184*1e4,
               'G_ref': 4.184,
               'H_ref': 4.184,
               'S_ref': 4.184,
               'V_ref': 0.1,
               'Cp_ref': 4.184,
               'omega0': 4.184*1e5}

for col in list(multipliers.keys()):
    dew[col] = dew[col] * multipliers[col]

dew

,name,symbol,formula,G_ref,H_ref,S_ref,V_ref,Cp_ref,omega0,z,...,prd_ac,compl,gas,a1,a2,a3,a4,c1,c2,comments
0,"acetate,aq",CH3COO-,C(2)H(3)O(2),-369321.680,-486097.12,86.19040,4.049993,26.408331,551534.88,-1,...,0,0,0,4.191531,1175.285600,28.116480,-121089.144000,110.039200,-161502.400000,"revised January 26th, 2016; new a1 value from ..."
1,"acetic-acid,aq",CH3COOH,C(2)H(4)O(2),-396475.840,-485762.40,178.65680,5.201002,171.893580,-31798.40,0,...,0,0,0,4.811600,2301.200000,6.987280,-120080.800000,187.861600,-110039.200000,Plyasunov & Shock (2001)
2,Ag+,Ag(+),Ag(1),77098.568,105750.60,73.38736,-0.080008,33.052241,90374.40,1,...,1,0,0,0.856222,-1835.819346,29.364018,-108684.082816,53.497768,-59640.249478,Shock & Helgeson (1988) revised with new predi...
3,"AgCl,aq",AgCl(0),Ag(1)Cl(1),-73010.800,-76441.68,142.67440,2.700000,28.033192,-15899.20,0,...,1,1,0,2.822461,895.198426,12.736460,-119974.110301,40.765485,-69864.051884,Sverjensky et al. (1997) with new V greater th...
4,AgCl2-,AgCl2(-),Ag(1)Cl(2),-215727.040,-255767.92,196.64800,5.300000,32.216365,383630.96,-1,...,1,1,0,5.126791,3089.149764,-5.659614,-129043.905131,80.025136,-61342.929492,Sverjensky et al. (1997) with new V greater th...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,Yb+3,Yb(+3),Yb(1),-640152.000,-670695.20,-238.06960,-4.449999,-152.296070,1022695.12,3,...,1,0,0,-3.859039,-599.364582,60.804063,-113795.586832,30.765404,-437194.758120,Shock & Helgeson (1988) revised with new predi...
224,Zn+2,Zn(+2),Zn(1),-147276.800,-153385.44,-109.62080,-2.430009,-26.360379,609776.16,2,...,1,0,0,-0.593050,-3965.892286,41.746694,-99878.361272,66.529939,-180663.755263,Shock & Helgeson (1988) revised with new predi...
225,ZnCl+,ZnCl(+),Zn(1)Cl(1),-279700.400,-277148.16,96.23200,0.480000,83.261218,84726.00,1,...,1,1,0,1.066810,-776.354860,26.752275,-113063.908995,82.402946,42635.437729,Sverjensky et al. (1997) with revised a1 equal...
226,"ZnCl2,aq",ZnCl2(0),Zn(1)Cl(2),-411287.200,-456390.72,113.09352,2.600000,145.017864,-15899.20,0,...,1,1,0,2.741203,817.833260,13.385159,-119654.282705,109.325714,168433.725945,Sverjensky et al. (1997) with revised a1 equal...


Create a function to generate the parameter dictionary required for each phase by the coder module. Create a name for the species that can be used in filenames.

In [6]:
def HKF_params(species='H+',Formula='', z=0.0,
                   G_ref=0.0, S_ref=0.0, V_ref=0.0, Cp_ref = 0.0,
                   a1=0.0, a2=0.0, a3=0.0, a4=0.0,
                   c1=0.0, c2=0.0, omega0=0.0,
                   theta=228.0, Psi=2600.0, eta=694657.0, rH=3.082,
                   T_r=298.15, P_r=1.0,
                   compl=0,gas=0,prd_ac=0,prd_omega=0):
    
    species_name = species
    
    if species_name[-1] == '-':
        species_name = species_name[:-1]+'_n'
    if species_name[-2] == '-':
        species_name = species_name[:-2]+'_n'+species_name[-1]
    species_name = species_name.replace('-','_')
    species_name = species_name.replace('+','_p')
    species_name = species_name.replace('(','_l_')
    species_name = species_name.replace(')','_r_')
        
    species_name = species_name.replace(',','_')
    
    param_dict = {'Phase': species_name,
                  'Formula': Formula,
                  'G_ref':G_ref,
                  'S_ref':S_ref,
                  'v_ref':V_ref,
                  'C_p_ref':Cp_ref,
                  'a1':a1,
                  'a2':a2,
                  'a3':a3,
                  'a4':a4,
                  'c1':c1,
                  'c2':c2,
                  'omega0':omega0,
                  'complex':int(compl),
                  'gas':int(gas),
                  'prd_ac':int(prd_ac),
                  'prd_omega':int(prd_omega),
                  'theta':theta,
                  'Psi':Psi,
                  'eta':eta,
                  'rH':rH,
                  'z':z,
                  'T_r':T_r,
                  'P_r':P_r}
    return param_dict

## Create objects for each DEW species
First, create a dictionary of phase parameters for each species

In [7]:
phase_params = {}
for i, row in dew.iterrows():
    phase_params[row['name']] = HKF_params(species=row['name'],Formula=row['formula'], z=row['z'],
                   G_ref=row['G_ref'], S_ref=row['S_ref'], V_ref=row['V_ref'], Cp_ref = row['Cp_ref'],
                   a1=row['a1'], a2=row['a2'], a3=row['a3'], a4=row['a4'],
                   c1=row['c1'], c2=row['c2'], omega0=row['omega0'],
                   compl=row['compl'],gas=row['gas'],prd_ac=row['prd_ac'],prd_omega=row['prd_omega'])

Build the database, including generating the coder files

In [8]:
mod_name = 'dew2019'
modelDB, codermod = make_custom_database(mod_name,phase_params)

/Users/simonmatthews/Dropbox/projects/ENKI-DEW/eq3wrap/working
Creating (once only) generic fast model code file string
Creating (once only) generic model fast code template include file string
Creating (once only) generic model fast code template code file string
Creating include file ...
... done!
Creating code file ...
... done
Writing include file to working directory ...
Writing code file to working directory ...
Writing pyxbld file to working directory ...
writing pyx file to working directory ...
Compiling code and Python bindings ...
Success! Import the module named  dew2019
Creating include file ...
... done!
Creating code file ...
... done
Writing include file to working directory ...
Writing code file to working directory ...
Writing pyxbld file to working directory ...
writing pyx file to working directory ...
Compiling code and Python bindings ...
Success! Import the module named  dew2019
Creating include file ...
... done!
Creating code file ...
... done
Writing include f

/Users/simonmatthews/opt/anaconda3/lib/python3.7/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /Users/simonmatthews/Dropbox/projects/ENKI-DEW/eq3wrap/working/dew2019.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
/Users/simonmatthews/opt/anaconda3/lib/python3.7/site-packages/thermoengine/core.py:468: RuntimeWarning: divide by zero encountered in double_scalars
  mol_Fe_oxide = Fe_remain/(2*frac_Fe2O3 + 1*frac_FeO)


/Users/simonmatthews/Dropbox/projects/ENKI-DEW/eq3wrap


## Check database
I'm not sure putting all the species into a database is the right thing to do since they're not individual phases. Consider this later once the necessary functionality has been identified.

In [9]:
modelDB.phase_info

,abbrev,phase_name,formula,phase_type,endmember_num
0,Liq,Liquid,,solution,15
1,"acetate,aq",acetate_aq,C2H3O2,pure,1
2,"acetic-acid,aq",acetic_acid_aq,C2H4O2,pure,1
3,Ag+,Ag_p,Ag,pure,1
4,"AgCl,aq",AgCl_aq,AgCl,pure,1
...,...,...,...,...,...
225,Zn+2,Zn_p2,Zn,pure,1
226,ZnCl+,ZnCl_p,ZnCl,pure,1
227,"ZnCl2,aq",ZnCl2_aq,ZnCl2,pure,1
228,ZnCl3-,ZnCl3_n,ZnCl3,pure,1


## Save output phases

In [10]:
output = {}
for phsnm in phase_params:
    iphs = modelDB.get_phase(phsnm)
    output[phsnm] = iphs
output

{'acetate,aq': <thermoengine.phases.PurePhase at 0x1a3b900dd0>,
 'acetic-acid,aq': <thermoengine.phases.PurePhase at 0x1a3b900cd0>,
 'Ag+': <thermoengine.phases.PurePhase at 0x1a36868450>,
 'AgCl,aq': <thermoengine.phases.PurePhase at 0x1a3b900e10>,
 'AgCl2-': <thermoengine.phases.PurePhase at 0x1a368682d0>,
 'Al+3': <thermoengine.phases.PurePhase at 0x1a30c4e1d0>,
 'AlO2-': <thermoengine.phases.PurePhase at 0x1a3b900990>,
 'AlO2(SiO2)-': <thermoengine.phases.PurePhase at 0x1a3b900f50>,
 'Ar,aq': <thermoengine.phases.PurePhase at 0x1a3b900110>,
 'Au+': <thermoengine.phases.PurePhase at 0x1a3b900290>,
 'Au+3': <thermoengine.phases.PurePhase at 0x1a30c8a090>,
 'B(OH)3,aq': <thermoengine.phases.PurePhase at 0x1a3b900f90>,
 'Ba+2': <thermoengine.phases.PurePhase at 0x1a3b900590>,
 'BaCl+': <thermoengine.phases.PurePhase at 0x1a3b900310>,
 'Be+2': <thermoengine.phases.PurePhase at 0x1a36868790>,
 'benzene,aq': <thermoengine.phases.PurePhase at 0x1a30c4e9d0>,
 'BO(OH),aq': <thermoengine.phas

In [11]:
with open('DEW2019.pkl','wb') as file:
    dill.dump(output, file)
    file.close()